In [1]:
import networkx as nx
import niche
import csv
import random
import numpy as np
out_location = "paper_webs_0511"

/Users/luva6824/jhub_venv/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [4]:
# Generate final metrics for plotting and save

def get_intermediate_ids(net):
    int_species = []
    for node in net.nodes():
        
        #top predator would have no outgoing edges
        out_neighs = []
        out_neigh_list = list(net.out_edges([node]))
        for out_neigh in out_neigh_list:
            out_neighs.append(out_neigh[1])
            
        in_neighs = []
        in_neigh_list = list(net.in_edges([node]))
        for in_neigh in in_neigh_list: #curr_web.in_degree(node):
            in_neighs.append(in_neigh[0])
            
        if len(out_neighs) > 0 and len(in_neighs) > 0:
            int_species.append(node)
            
    return int_species

def get_basal_ids(net):
    basal_species = []
    
    for node in net.nodes():
        
        out_neighs = []
        out_neigh_list = list(net.out_edges([node]))
        for out_neigh in out_neigh_list:
            out_neighs.append(out_neigh[1])
            
        in_neighs = []
        in_neigh_list = list(net.in_edges([node]))
        for in_neigh in in_neigh_list: #curr_web.in_degree(node):
            in_neighs.append(in_neigh[0])
            
        if len(out_neighs) > 0 and len(in_neighs) == 0:
            basal_species.append(node)
            
    return basal_species
    
    
# four metrics of interest returned:
# intro_metric1 - final biomass of introduced species 1
# intro_metric2 - final biomass of introduced species 2
# es_metric - % change in es amount
# bio_metric - fraction resident species remaining
def get_metrics(web_init, web_final, b_init,b_final,inv_id1,inv_id2,es_fn):
        
    #based off of aggregate of intermediate species biomasses
    #init_int = get_intermediate_ids(web_init)
    init_int = get_basal_ids(web_init)
    b_total_init = 0
    for node in init_int:
        b_total_init += b_init[node]
    
    initial_es = es_fn(b_total_init)
    initial_bio = len(b_init.keys())
    
    inv_adj = 0
    intro_metric1 = 0
    intro_metric2 = 0
    if inv_id1 and inv_id1 in b_final:
        intro_metric1 = b_final[inv_id1]
        inv_adj = inv_adj - 1
    if inv_id2 and inv_id2 in b_final:
        intro_metric2 = b_final[inv_id2]
        inv_adj = inv_adj - 1
    
    #final_int = get_intermediate_ids(web_final)
    final_int = get_basal_ids(web_final)
    b_total_final = 0
    for node in final_int:
        if node != inv_id1 and node != inv_id2:
            b_total_final += b_final[node]
    
    final_es = es_fn(b_total_final)
    final_bio = len(b_final.keys()) + inv_adj # subtract out the invader counts
    
    es_metric = (final_es - initial_es)/initial_es*100
    bio_metric = final_bio/initial_bio
    
    return intro_metric1, intro_metric2, es_metric, bio_metric
    
# get es metric assuming it's provided by one species
def get_es_spec(b_init, b_final, es_spec):
    
    initial_es = es_linear(b_init[es_spec])
    
    if es_spec in b_final:
        final_es = es_linear(b_final[es_spec])
    else:
        final_es = 0    
    
    es_metric = (final_es - initial_es)/initial_es*100
    
    return es_metric
    
#maps the value of species biomass to es value
def es_linear(b_val):
    return b_val*0.5


In [6]:
# Generate final metrics for plotting and save to file 

fif_webs_file = open('./' + out_location + '/over15_webs.txt','r')
ct_file = open('./' + out_location + '/counterfactual_results_newmet.csv','w')
ct_writer = csv.writer(ct_file,dialect="excel")
ct_writer.writerow(["web_id","intro1_metric","ïntro2_metric","es_metric","bio_metric"])

es_file1 = open('./' + out_location + '/all_es_res_ct.csv','w')
esw_ct = csv.writer(es_file1)
esw_ct.writerow(["web_id","node_id","es_metric"])
es_file2 = open('./' + out_location + '/all_es_res_inv1.csv','w')
esw_inv1 = csv.writer(es_file2)
esw_inv1.writerow(["web_id","node_id","es_metric"])
es_file3 = open('./' + out_location + '/all_es_res_inv2.csv','w')
esw_inv2 = csv.writer(es_file3)
esw_inv2.writerow(["web_id","node_id","es_metric"])
es_file4 = open('./' + out_location + '/all_es_res_invb.csv','w')
esw_invb = csv.writer(es_file4)
esw_invb.writerow(["web_id","node_id","es_metric"])

for row in fif_webs_file:
    i = int(row)
    
    #initial biomass vector for web (before introduction, after run to equilibrium)
    webi,_,_,_,b_init_web = niche.read_web_from_file("./" + out_location + "/web_" + str(i) + "_2000")
        
    # save metrics for counterfactual dynamics (4000 total timesteps with no introduction)
    webf,_,_,_,b_final_ct = niche.read_web_from_file("./" + out_location + "/web_" + str(i) + "_2000_2000")
    intro1_ct, intro2_ct, es_ct, bio_ct = get_metrics(webi,webf,b_init_web,b_final_ct,None,None,es_linear)
    
    ct_writer.writerow([i, intro1_ct, intro2_ct, es_ct, bio_ct])
    for node in get_basal_ids(webi):
        esw_ct.writerow([i,node,get_es_spec(b_init_web,b_final_ct,node)])
        
    # invader 1
    inv_1_info = open('./' + out_location + '/invader_1.csv','r')
    inv1_reader = csv.reader(inv_1_info)
    next(inv1_reader)
    #object for temporary saving
    es_nodes = {}
    inv_1_res_dict = {}
    for line in inv1_reader:
        inv_id = int(line[0])
        #invaded web with additional 2000 timesteps after invasion
        webf,_,_,_,b_final_1 = niche.read_web_from_file("./" + out_location + "/web_" + str(i) + "_inv/one/web_" + str(i) + \
                                                     "_inv_" + str(inv_id) + "_2000")
        intro1_inv, intro2_inv, es_inv, bio_inv = get_metrics(webi,webf,b_init_web,b_final_1,inv_id,None,es_linear)
        inv_1_res_dict[inv_id] = [intro1_inv, intro2_inv, es_inv, bio_inv]
        
        for node in webi.nodes():
            if node not in es_nodes:
                es_nodes[node] = [get_es_spec(b_init_web,b_final_1,node)]
            else:
                es_nodes[node].append(get_es_spec(b_init_web,b_final_1,node))
            
    inv_1_info.close()

    #write invader 1 results to file 
    inv_1_res = open('./' + out_location + '/web_' + str(i) + '_invader_1_newmet.csv','w')
    inv1_writer = csv.writer(inv_1_res,dialect="excel")
    inv1_writer.writerow(['inv_id','intro1_metric','intro2_metric','es_metric','bio_metric'])
    for key in inv_1_res_dict:
        inv1_writer.writerow([key] + inv_1_res_dict[key])
    inv_1_res.close()
    
    for node in get_basal_ids(webi):
        esw_inv1.writerow([i,node,np.mean(es_nodes[node])])

    # invader 2
    inv_2_info = open('./' + out_location + '/invader_2.csv','r')
    inv2_reader = csv.reader(inv_2_info)
    next(inv2_reader)
    #object for temporary saving
    inv_2_res_dict = {}
    es_nodes = {}
    for line in inv2_reader:
        inv_id = int(line[0])
        #invaded web with additional 2000 timesteps after invasion
        webf,_,_,_,b_final_2 = niche.read_web_from_file("./" + out_location + "/web_" + str(i) + "_inv/two/web_" + str(i) + \
                                                     "_inv_" + str(inv_id) + "_2000")
        intro1_inv, intro2_inv, es_inv, bio_inv = get_metrics(webi,webf,b_init_web,b_final_2,None,inv_id,es_linear)
        inv_2_res_dict[inv_id] = [intro1_inv, intro2_inv, es_inv, bio_inv]
        
        for node in webi.nodes():
            if node not in es_nodes:
                es_nodes[node] = [get_es_spec(b_init_web,b_final_2,node)]
            else:
                es_nodes[node].append(get_es_spec(b_init_web,b_final_2,node))
                
    inv_2_info.close()

    #write invader 2 results to file 
    inv_2_res = open('./' + out_location + '/web_' + str(i) + '_invader_2_newmet.csv','w')
    inv2_writer = csv.writer(inv_2_res,dialect="excel")
    inv2_writer.writerow(['inv_id','intro1_metric','intro2_metric','es_metric','bio_metric'])
    for key in inv_2_res_dict:
        inv2_writer.writerow([key] + inv_2_res_dict[key])
    inv_2_res.close()
    
    for node in get_basal_ids(webi):
        esw_inv2.writerow([i,node,np.mean(es_nodes[node])])
    
    # both invaders 
    inv_both_info = open('./' + out_location + '/invader_both.csv','r')
    inv_both_reader = csv.reader(inv_both_info)
    next(inv_both_reader)
    #object for temporary saving
    inv_both_res_dict = {}
    es_nodes = {}
    for line in inv_both_reader:
        inv1_id = int(line[0])
        inv2_id = int(line[1])
        #invaded web with additional 2000 timesteps after invasion
        webf,_,_,_,b_final_b = niche.read_web_from_file("./" + out_location + "/web_" + str(i) + "_inv/both/web_" + str(i) + "_inv_" + str(inv1_id) + '_' + str(inv2_id) + "_2000")
        intro1_inv, intro2_inv, es_inv, bio_inv = get_metrics(webi,webf,b_init_web,b_final_b,inv1_id,inv2_id,es_linear)
        inv_both_res_dict[(inv1_id,inv2_id)] = [intro1_inv, intro2_inv, es_inv, bio_inv]
        
        for node in webi.nodes():
            if node not in es_nodes:
                es_nodes[node] = [get_es_spec(b_init_web,b_final_b,node)]
            else:
                es_nodes[node].append(get_es_spec(b_init_web,b_final_b,node))
        
    inv_both_info.close()

    #write both invader results to file 
    inv_both_res = open('./' + out_location + '/web_' + str(i) + '_invader_both_newmet.csv','w')
    inv_both_writer = csv.writer(inv_both_res,dialect="excel")
    inv_both_writer.writerow(['inv_id1','inv_id2','intro1_metric','intro2_metric','es_metric','bio_metric'])
    for key in inv_both_res_dict:
        inv_both_writer.writerow([key[0],key[1]] + inv_both_res_dict[key])
    inv_both_res.close()
    
    for node in get_basal_ids(webi):
        esw_invb.writerow([i,node,np.mean(es_nodes[node])])

fif_webs_file.close()
ct_file.close()
es_file1.close()
es_file2.close()
es_file3.close()
es_file4.close()